In [1]:
# Python API Homework - Part II - VacationPy
# UT-TOR-DATA-PT-01-2020-U-C Assignment #6
# (c) Boris Smirnov

# VacationPy
----


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

# Cities data file
cities_fname = '../output_data/cities.csv'


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv(cities_fname)
cities_df.dtypes

Country        object
City           object
Latitude      float64
Longitude     float64
Belt           object
Date            int64
Max Temp      float64
Wind Speed    float64
Cloudiness      int64
Humidity        int64
Lon           float64
Lat           float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Latitude", "Longitude"]]

In [5]:
humidity = cities_df['Humidity']
humidity_fig = gmaps.figure()
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=95)
humidity_fig.add_layer(humidity_layer)
humidity_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Preferable weather conditions
max_temperature = 28 # C ~ 82F
min_temperature = 23 # C ~ 73F
max_wind_speed = 4.5 # m/s ~ 10mph
max_cloudiness = 10

In [7]:
# Cities with desireabale weather conditions
best_cities_df = cities_df.loc[
    (cities_df['Max Temp'] <= max_temperature) & (cities_df['Max Temp'] >= min_temperature) & \
    (cities_df['Wind Speed'] <= max_wind_speed) & (cities_df['Cloudiness'] <= max_cloudiness),
:].copy()
best_cities_df

,Country,City,Latitude,Longitude,Belt,Date,Max Temp,Wind Speed,Cloudiness,Humidity,Lon,Lat
51,in,kiraoli,27.150000,77.783333,"(20, 30]",1584020596,23.18,1.94,10,26,77.78,27.15
53,np,rajbiraj,26.533333,86.750000,"(20, 30]",1584020598,26.46,2.11,0,37,86.75,26.53
54,in,chhatapur,26.216667,87.016667,"(20, 30]",1584020599,27.64,2.62,0,36,87.02,26.22
59,in,akaltara,22.016667,82.433333,"(20, 30]",1584020606,26.94,4.21,6,61,82.43,22.02
62,in,utran,21.233333,72.866667,"(20, 30]",1584020610,24.00,2.60,0,47,72.87,21.23
76,in,farakka,24.816667,87.900000,"(20, 30]",1584020628,27.55,0.86,0,36,87.90,24.82
77,in,rajsamand,25.066667,73.883333,"(20, 30]",1584020630,25.00,4.10,0,27,73.88,25.07
78,in,maghar,26.757222,83.127778,"(20, 30]",1584020631,25.23,2.26,0,39,83.13,26.76
85,in,rahatgarh,23.783333,78.366667,"(20, 30]",1584020640,24.91,2.65,4,31,78.37,23.78
87,in,supaul,26.116667,86.600000,"(20, 30]",1584020475,26.72,3.37,0,34,86.60,26.12


### Hotel Map
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

parameters = {
    'key': g_key,
    'language': 'en',
    'radius': '10000', # When 5000, 6 places miss hotel info. When 1000, only 4 and the hotels have better ratings
    'type': 'lodging',
    'keyword': 'hotel'
}

In [9]:
best_cities_df.loc[:, 'Hotel Name'] = ''
best_cities_df.loc[:, 'Rating'] = ''

for index, city in best_cities_df.iterrows():

    # City's coordinates from the database
    lat = city["Latitude"]
    lng = city["Longitude"]

    # Google Places API Nearby Search
    parameters["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=parameters)
    if response.status_code != 200:
        best_cities_df.loc[idx, "Hotel Name"] = np.NaN
        best_cities_df.loc[idx, "Rating"] = np.NaN
        print(f"Google Places query failed for index {index} ... skipping.")
        continue
        
    hotels = response.json()
    #print(json.dumps(hotels, indent=4))
    #pause = input('Press Enter to continue')

    try:
        best_cities_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        best_cities_df.loc[index, "Rating"] = hotels["results"][0]["rating"]
    except:
        if best_cities_df.loc[index, "Hotel Name"] == "": best_cities_df.loc[index, "Hotel Name"] = np.NaN
        if best_cities_df.loc[index, "Rating"] == "": best_cities_df.loc[index, "Rating"] = np.NaN
        print(f"Missing field/result for index {index} ... skipping.")

Missing field/result for index 59 ... skipping.
Missing field/result for index 181 ... skipping.
Missing field/result for index 358 ... skipping.
Missing field/result for index 456 ... skipping.


In [10]:
# Let's drop those cities that don't have hotels nearby
best_cities_df.dropna(inplace=True)
best_cities_df['Country'] = best_cities_df['Country'].str.upper()
best_cities_df['City'] = best_cities_df['City'].str.title()
best_cities_df

,Country,City,Latitude,Longitude,Belt,Date,Max Temp,Wind Speed,Cloudiness,Humidity,Lon,Lat,Hotel Name,Rating
51,IN,Kiraoli,27.150000,77.783333,"(20, 30]",1584020596,23.18,1.94,10,26,77.78,27.15,OYO 44134 Hotel Radhika Resort,4.4
53,NP,Rajbiraj,26.533333,86.750000,"(20, 30]",1584020598,26.46,2.11,0,37,86.75,26.53,Sagar Lodge,3.2
54,IN,Chhatapur,26.216667,87.016667,"(20, 30]",1584020599,27.64,2.62,0,36,87.02,26.22,D.K. Sweets Corner,5
62,IN,Utran,21.233333,72.866667,"(20, 30]",1584020610,24.00,2.60,0,47,72.87,21.23,Surat Marriott Hotel,4.5
76,IN,Farakka,24.816667,87.900000,"(20, 30]",1584020628,27.55,0.86,0,36,87.90,24.82,Ganga Bhavan Guest House,4.4
77,IN,Rajsamand,25.066667,73.883333,"(20, 30]",1584020630,25.00,4.10,0,27,73.88,25.07,Hotel Shreenath,3.8
78,IN,Maghar,26.757222,83.127778,"(20, 30]",1584020631,25.23,2.26,0,39,83.13,26.76,Hotel Soni International,3.7
85,IN,Rahatgarh,23.783333,78.366667,"(20, 30]",1584020640,24.91,2.65,4,31,78.37,23.78,Arvind Kushwaha Kirana Store,0
87,IN,Supaul,26.116667,86.600000,"(20, 30]",1584020475,26.72,3.37,0,34,86.60,26.12,Hotel R K Palace,3.9
151,PH,Bunga,10.773100,124.784700,"(10, 20]",1584020727,25.53,4.44,1,83,124.78,10.77,"Red Palm Inn - Candadam, Baybay City, Leyte",4.4


In [11]:
# Added hotel rating to the infobox

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in best_cities_df.iterrows()]
locations = best_cities_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
humidity_fig.add_layer(markers)
humidity_fig

Figure(layout=FigureLayout(height='420px'))